In [28]:
# Install and import necessary libraries
!pip install -q openai
!pip install --upgrade openai
import openai
import google.colab.userdata as userdata
import json

In [29]:
# Your Python tools as regular functions.
def say_hi(query: str) -> str:
    """
    Use this tool when the user greets you, says hello, or starts a conversation.
    """
    return "Hi!"

def say_bye(query: str) -> str:
    """
    Use this tool when the user wants to end the conversation or says goodbye.
    """
    return "Bye!"

def hi_follow_up(query: str) -> str:
    """
    Use this tool as follow-up responses when the user greets you, says hello, or starts a conversation.
    """
    return "How are you?"

def bye_follow_up(query: str) -> str:
    """
    Use this tool as follow-up responses when the user wants to end the conversation or says goodbye.
    """
    return "Have a great day!"


In [30]:
# Configure with your API key
openai.api_key = userdata.get("OPENAI_API_KEY")

In [31]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=userdata.get("OPENAI_API_KEY"),
    api_version="2024-12-01-preview",   # check latest supported version
    azure_endpoint=userdata.get("AZURE_OPENAI_ENDPOINT"),
)

In [32]:
def get_azure_response(messages,user_input, tools=None ):
    try:
        response = client.chat.completions.create(
            model=userdata.get("AZURE_OPENAI_DEPLOYMENT"),  # deployment name
            messages=[
            {"role": "system", "content": messages},
            {"role": "user", "content": user_input},
        ],
            tools=tools,
            tool_choice="auto" if tools else None,
        )
        return response.choices[0].message
    except Exception as e:
        print(f"Azure OpenAI API Error: {e}")
        return None

In [33]:
# A dictionary to map tool names to the actual functions.
tools_map = {
    "say_hi": say_hi,
    "say_bye": say_bye
}

follow_up_tools_map = {
    "hi_follow_up": hi_follow_up,
    "bye_follow_up": bye_follow_up,
}

# System Instruction
messages1 =  '''
You are an assistant designed to use tools to respond to user requests.
Your primary task is to carefully analyze the user's input and determine which of the available tools is most appropriate to accomplish their goal.

If a tool is a perfect match, call it with the correct parameters.
If no tool is suitable for the user's request, do not aexttempt to call any tool. Instead, respond directly with the exact phrase: 'You can do better!'
'''

messages2 =  '''
You are a follow-up assistant agent.
Your job is to look at the name of the tool the previous agent called and, based on that name, select the appropriate follow-up tool to call with suggested followup response as a arguments.
For example, if the previous tool was say_hi, call hi_follow_up. If it was say_bye, call bye_follow_up.
'''

# The conversational loop
print("Agent is ready. Type 'exit' to end the conversation.")

# The conversational loop
for turn in range(10): # A for loop to limit the number of turns for this example
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        print("Agent: Goodbye!")
        break

    # Tool definitions for Agent 1
    tools_agent_1 = [
        {
            "type": "function",
            "function": {
                "name": "say_hi",
                "description": "Use this tool when the user greets you, says hello, or starts a conversation.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The user's greeting.",
                        }
                    },
                    "required": ["query"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "say_bye",
                "description": "Use this tool when the user wants to end the conversation or says goodbye.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The user's farewell.",
                        }
                    },
                    "required": ["query"],
                },
            },
        },
    ]

    # Get response from Agent 1
    response = get_azure_response(messages1, user_input, tools_agent_1)
    if not response:
        continue

    # Handle tool calls
    if response.tool_calls:
        tool_call = response.tool_calls[0]
        tool_name = tool_call.function.name
        tool_args = json.loads(tool_call.function.arguments)

        print(f"Agent 1 chose to call tool: {tool_name} with arguments: {tool_args}")

        # Execute the tool
        tool_result = tools_map[tool_name](**tool_args)

        print(f"Agent_1: {tool_result}")

        # Tool definitions for Agent 2 (follow-up)
        tools_agent_2 = [
            {
                "type": "function",
                "function": {
                    "name": "hi_follow_up",
                    "description": "Use this tool as a follow-up response when the user greets you.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "The tool name from the previous agent call.",
                            }
                        },
                        "required": ["query"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "bye_follow_up",
                    "description": "Use this tool as a follow-up response when the user says goodbye.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "The tool name from the previous agent call.",
                            }
                        },
                        "required": ["query"],
                    },
                },
            },
        ]

        # Determine the follow-up tool to call
        follow_up_response = get_azure_response(
            messages2,
            tool_name,
            tools_agent_2
        )

        if follow_up_response and follow_up_response.tool_calls:
            follow_up_tool_call = follow_up_response.tool_calls[0]
            follow_up_tool_name = follow_up_tool_call.function.name
            follow_up_tool_args = json.loads(follow_up_tool_call.function.arguments)

            print(f"Agent 2 chose to call tool: {follow_up_tool_name} with arguments: {follow_up_tool_args}")

            # Execute the follow-up tool
            follow_up_result = follow_up_tools_map[follow_up_tool_name](**follow_up_tool_args)

            print(f"Agent_2: {follow_up_result}")
        else:
            print("Agent_2: No follow-up tool called.")

    else:
        # If no tool was called, the model responded with text
        print(f"Agent: {response.content}")

Agent is ready. Type 'exit' to end the conversation.
You: Hi
Agent 1 chose to call tool: say_hi with arguments: {'query': 'Hi'}
Agent_1: Hi!
Agent 2 chose to call tool: hi_follow_up with arguments: {'query': 'Hello! How can I help you today?'}
Agent_2: How are you?
You: Bye
Agent 1 chose to call tool: say_bye with arguments: {'query': 'Bye'}
Agent_1: Bye!
Agent 2 chose to call tool: bye_follow_up with arguments: {'query': 'say_bye'}
Agent_2: Have a great day!
You: Sky is green
Agent: You can do better!
You: Goodbye
Agent 1 chose to call tool: say_bye with arguments: {'query': 'Goodbye'}
Agent_1: Bye!
Agent 2 chose to call tool: bye_follow_up with arguments: {'query': 'Goodbye! It was nice talking to you. Take care and see you next time!'}
Agent_2: Have a great day!
You: Exit
Agent: Goodbye!
